In [1]:
from datetime import timedelta, datetime
from pathlib import Path

import geopandas as gp
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import fiona
import pandas as pd

from scripts.utility import *
from scripts.map_utility import *
from scripts.dates import (
    bumbershoot_dates,
    siff_dates,
    hempfest_dates,
    pride_dates,
    summer_dates,
    all_events,
)

# set data paths
nhood_path = "data/intermediate/nhood/"
finished_path = "data/finished/nhood"
seattle_neigh_json = "/Users/dmoton/udacity/dsnd-moton/airbnb_analysis/data/raw/neighborhoods.geojson"

In [2]:
%load_ext autoreload
%autoreload 2

## Load stuff

In [3]:
price_metrics_dict = dict()
occp_metrics_dict = dict()
for event_name in all_events.keys():
    occp_path = Path(nhood_path + event_name + "_occpchange.csv")
    price_path = Path(nhood_path + event_name + "_pricechange.csv")
    print(occp_path.stem, price_path.stem)
    price_metrics_dict[price_path.stem] = pd.read_csv(price_path)
    occp_metrics_dict[occp_path.stem] = pd.read_csv(occp_path)                                        

bumbershoot_occpchange bumbershoot_pricechange
siff_occpchange siff_pricechange
hempfest_occpchange hempfest_pricechange
pride_occpchange pride_pricechange


In [4]:
seattle_neigh = gp.read_file(seattle_neigh_json)

In [5]:
price_metrics_dict['bumbershoot_pricechange']

,Unnamed: 0,neighborhood,pct_change_price,event_mean_price
0,0,Ballard,-0.215232,125.506250
1,1,Beacon Hill,-0.191642,129.644048
2,2,Capitol Hill,0.240371,168.600294
3,3,Cascade,0.046832,220.941667
4,4,Central Area,-0.038991,165.814103
5,5,Delridge,0.031353,147.262500
6,6,Downtown,0.464432,190.188301
7,7,Lake City,-0.429127,119.566667
8,8,Magnolia,-0.152811,157.625000
9,9,Northgate,-0.371910,177.061111


## Plot price and availability stuff

In [28]:
occp_data = dict()
for event, avail in occp_metrics_dict.items():
    merged = seattle_neigh.merge(
        avail,
        left_on='nhood',
        right_on='neighborhood',
        how='right'
    ).drop_duplicates(['nhood'])
    merged_plot = merged.cx[-122.42:-122.25, 47.50:47.70]
    
    merged_plot_pts = merged_plot.copy()
    merged_plot_pts['geometry'] = merged_plot_pts['geometry'].centroid
        
    occp_data[event.split('_')[0]] = px.scatter_mapbox(
        merged_plot_pts,
        lat=merged_plot_pts.geometry.y,
        lon=merged_plot_pts.geometry.x,
        size='count',
        color='occp_perc',
        hover_name='nhood',
        color_continuous_scale='Inferno',
        range_color=[0,1],
        size_max=20,
        zoom=14,
        title="AirBnB Occupancy By Neighborhood",
        hover_data={
            'count': False,
            'occp_perc': False,
            'pct_change_occp': False,
            'Total Listings': merged_plot_pts['count'].to_list(), 
            'Change in Occupany': format_percent_field(merged_plot_pts['pct_change_occp'], to_percent=True).to_list(),
            'Occupancy': format_percent_field(merged_plot_pts['occp_perc'], to_percent=True).to_list(),
        }
    )  

In [31]:
avail_fig = create_map_dropdown(occp_data)

In [32]:
avail_fig.show()

In [40]:
price_data = dict()
for event, price in price_metrics_dict.items():
    merged = seattle_neigh.merge(
        price,
        left_on='nhood',
        right_on='neighborhood',
        how='right'
    ).drop_duplicates(['nhood'])
    merged_plot = merged.cx[-122.42:-122.25, 47.50:47.70]
    
    merged_plot_pts = merged_plot.copy()
    merged_plot_pts['geometry'] = merged_plot_pts['geometry'].centroid
        
    price_data[event.split('_')[0]] = px.scatter_mapbox(
        merged_plot_pts,
        lat=merged_plot_pts.geometry.y,
        lon=merged_plot_pts.geometry.x,
        size='event_mean_price',
        color='pct_change_price',
        hover_name='nhood',
        color_continuous_scale='Inferno',
        range_color=[-0.5,2],
        size_max=20,
        zoom=14,
        title="Listing Price By Neighborhood",
#         hover_data={
#             'event_mean_price': False,
#             'pct_change_price': False,
#             'Average Listing Price': format_dollar_field(
#                 merged_plot_pts['event_mean_price'],
#                 to_dollar=True
#             ).to_list(),
#             'Change in Price': format_percent_field(
#                 merged_plot_pts['pct_change_price'],
#                 to_percent=True
#             ).to_list()
#         }
    )  

In [52]:
price_data['siff'].layout

Layout({
    'coloraxis': {'cmax': 2,
                  'cmin': -0.5,
                  'colorbar': {'title': {'text': 'pct_change_price'}},
                  'colorscale': [[0.0, '#000004'], [0.1111111111111111,
                                 '#1b0c41'], [0.2222222222222222, '#4a0c6b'],
                                 [0.3333333333333333, '#781c6d'],
                                 [0.4444444444444444, '#a52c60'],
                                 [0.5555555555555556, '#cf4446'],
                                 [0.6666666666666666, '#ed6925'],
                                 [0.7777777777777778, '#fb9b06'],
                                 [0.8888888888888888, '#f7d13d'], [1.0, '#fcffa4']]},
    'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
    'mapbox': {'center': {'lat': 47.60925937621736, 'lon': -122.33643510362207},
               'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
               'zoom': 14},
    'template': '...',
    'title': {'text': 'Listing Price B

In [70]:
f = go.Figure()

In [72]:
f.layout = price_data['siff'].layout

In [89]:
price_fig = create_map_dropdown(price_data)

In [90]:
price_fig.show()

In [55]:
import chart_studio 

username = 'culight' # your username
api_key = 'LmBrEGkmRz5uOW6ed6ad' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [13]:
import chart_studio.plotly as py

py.plot(price_fig, filename = 'price_byevent', auto_open=True)

'https://plotly.com/~culight/2/'